In [1]:
from backtesting import plot_backtest, st_buy_hold
import pandas as pd
df = pd.read_csv("DATASET.CSV")

# st_ma(df)
st_buy_hold(df)
df["PercentChange"] = df.Close.pct_change()
plot_backtest(df)


In [5]:
import qgrid
widget = qgrid.show_grid(df)
widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
import numpy as np

def plot_backtest(df):
    # poor man's backtesting
    # https://www.pythonforfinance.net/2017/02/21/intraday-stock-mean-reversion-trading-backtest-in-python-with-short-selling/
    df['MA'] = df['Close'].rolling(window=200).mean()
    # df['Stdev'] = df['Close'].rolling(window=90).std()
    df["PercentageChange"] = df.Close.pct_change()

    #create a column which holds a TRUE value if the gap down from previous day's low to next 
    #day's open is larger than the 90 day rolling standard deviation
    df['Buy1'] = df['Close'] < df['High'].shift(1)
    df['Buy2'] = df['Close'] > df['MA']
    df['Sell1'] = df['Close'] > df['Low'].shift(1)
    df['Sell2'] = df['Close'] < df['MA']
    df['BUY'] = df['Buy1'] & df['Buy2']
    df['SELL'] = df['Sell1'] & df['Sell2']
    # df['SELL'] = 0

    # his method?
    df['Rets'] = np.where(df['BUY'], df['PercentageChange'], 0)
    df['Rets'] = np.where(df['SELL'], -df['PercentageChange'], df['Rets'])

    # my method
    df['Holdings'] = np.where(df['BUY'], 1, None)
    df['Holdings'] = np.where(df['SELL'], -1, df['Holdings'])
    df.Holdings = df.Holdings.fillna(method="ffill")
    # df.Holdings = df.Holdings.cumsum()
    df['Rets'] = df.Holdings * df.PercentageChange
    # print (df.Holdings)

    # returns = df['Rets'].dropna().cumsum()
    returns = df['Rets'].cumsum()
    buys = df[df.BUY]
    # sells = df[df.SELL]

    # import plotly.express as px
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=2, cols=2, shared_xaxes=True)
    fig.add_scatter(x=df.Date, y=df.Holdings, name="Holdings", row=1, col=1)
    fig.add_scatter(x=df.Date, y=returns, name="Returns", row=1, col=1)

    fig.add_scatter(x=df.Date, y=df.Close, mode="lines", name="Close", row=2, col=1)
    fig.add_scatter(x=df.Date, y=df.MA, mode="lines", name="MA", row=2, col=1)
    fig.add_scatter(x=buys.Date, y=buys.Close, mode="markers", marker=dict(size=10, symbol=5, color="green"), name="Buys", row=2, col=1)
    fig.add_scatter(x=sells.Date, y=sells.Close, mode="markers", marker=dict(size=10, symbol=6, color="red"), name="Sells", row=2, col=1)
    fig.update_yaxes(type="log", row=2, col=1)
    # fig.update_layout(yaxis_type="log", row=2, col=1)
    fig.show()